# Le merge des Datasets

Le but de ce notebook est de réaliser le merging entre le dataset scrapé depuis le site [Caradisiac](https://www.caradisiac.com/) (Voir le notebook **1_WebScrapping** pour la méthode de scarping et **2_ScrapedData_cleaning** pour le preprocessing notamment de la colone caracteristique) et le dataset fourni dans le cadre de ce projet (Data_cars.csv).<br>

Pour le merge, nous avons essayé d'utiliser la librarie FuzzyWuzzy. Mais, elle est très chronophage vu les tailles de nos datasets. <br>
Ainsi, nous avons utilisé une méthode de NLP basée sur TF-IDF décrite dans [ici](https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536), 
 (Voir le [code](https://colab.research.google.com/drive/1qhBwDRitrgapNhyaHGxCW8uKK5SWJblW#scrollTo=xo-X_nds97UN&forceEdit=true&sandboxMode=true)).
 
La méthode basée sur TF-IDF a pris plus de 8 heures pour faire le matching. D'après la rérérence cette méthode est approximativement 66 000 fois plus rapide que le FuzzyWuzzy (nos tests sur quelques echantillons ont peut le confirmer)

In [33]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

import re

#!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

pd.set_option('max_colwidth', 10000)

In [3]:
%%HTML
<style>.dataframe th, td:first-child{background:#3f577c;font-family:monospace;color:white;border:3px solid white;
text-align:left !important;}#codex{float:right;}</style>

## 1. Import des datasets

In [5]:
datasets_path = r'../Datasets/'
scraped_data = pd.read_csv(datasets_path+'scraped_Car_versions.csv')
data = pd.read_csv(datasets_path+'Data_cars.csv')

/anaconda3/envs/Mask_RCNN_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (54,55,56,57,58,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 2. Création de la variable du merge 'version'

Pour data, on extrait d'abord la variable ***version*** du **Description**. Puis, on concatène les variables :  **Make**, **Model** et **version** afin d'avoir en sortie : **make_model_version**,<br>
pour scraped_data, on concatène les variables : **model** et **version_name** afin d'avoir en sortie: **model_version**,<br>

==> Ainsi, nous obtenons les variables **make_model_version** et **model_version** qui sont assez homogènes pour réaliser le matching entre les deux datasets.

In [6]:
data['version'] = data['Description'].str.extract(r'(\bversion\b.*?,)', expand=True)       
data['version'] = data['version'].str.replace("version: ", "")
data['version'] = data['version'].str.replace(",", "")

data['make_model_version'] = data['Make'] + " " + data['Model'] + " " + data['version']

scraped_data['model_version'] = scraped_data['model'] + " " +scraped_data['version_name']

# Suppression des modèles qui n'existent pas dans le dataframe 'data' (afin de réduire la taille du 'scraped_data')
makes = data['Make'].unique()
scraped_data_makes = scraped_data.loc[scraped_data['brand'].str.upper().isin(makes)]

In [13]:
data[['Make', 'Model','version', 'make_model_version']].head(1)

,Make,Model,version,make_model_version
0,PEUGEOT,308,1.6 HDI FAP 92CH BUSINESS 5P,PEUGEOT 308 1.6 HDI FAP 92CH BUSINESS 5P


In [14]:
scraped_data_makes[['model', 'version_name' ,'model_version']].head(1)

,model,version_name,model_version
0,abarth 124,II 1.4 TURBO 170,abarth 124 II 1.4 TURBO 170


## 3. Déclaration des fonctions de ngrams et GetNeaestN
Voir [ici](https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536) pour plus de détails.

In [15]:
def ngrams(string, n=3):
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = ["|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

###matching query:
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

## 4. Matching

In [17]:
# Vecorisation des données 
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
model_version_scraping = scraped_data_makes['model_version'].unique()
#org_names = data_test['make_model_version'].unique()
tfidf = vectorizer.fit_transform(model_version_scraping)

nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)

In [ ]:
unique_org = data['make_model_version'].values # colone à matcher avec 'model_version' du 'scraped_data'

# réalisation du neast N
distances, indices = getNearestN(unique_org)

unique_org = list(unique_org) #need to convert back to a list

# recherche des matches
matches = []

for i,j in enumerate(indices):
    temp = [round(distances[i][0],2), scraped_data_makes.values[j][0][64],unique_org[i]]
    matches.append(temp)

# Construction du dataframe des matches 
matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','scraped_version','Origional_data'])

**Plus de 8 heures d'exécution**

In [25]:
# Sauvegarde du matches
matches.to_csv(datasets_path+'matches.csv')

In [24]:
matches

,Match confidence (lower is better),scraped_version,Origional_data
0,0.76,peugeot 308 1.6 HDI 92 BUSINESS 5P,PEUGEOT 308 1.6 HDI FAP 92CH BUSINESS 5P
1,0.33,bmw x6 f16 (F16) XDRIVE30D 258 M SPORT BVA8,BMW X6 (F16) XDRIVE30D 258 M SPORT BVA8
2,0.48,audi rs6 (3e generation) avant III AVANT 4.0 T...,AUDI RS6 III AVANT 4.0 TFSI 560 QUATTRO TIPTRONIC
3,0.00,ford ka 1.3 70 FUN,FORD KA 1.3 70 FUN
4,0.46,fiat tipo 2 sw II SW 1.4 95 POP,FIAT TIPO II SW 1.4 95 POP
...,...,...,...
166690,0.50,audi a4 (4e generation) avant IV (2) AVANT 2.0...,AUDI A4 IV (2) AVANT 2.0 TDI 177 BUSINESS LINE...
166691,0.41,volkswagen golf 7 VII 2.0 TDI 150 BLUEMOTION T...,VOLKSWAGEN GOLF VII 2.0 TDI 150 BLUEMOTION TEC...
166692,0.34,ford fiesta 5 V (2) 1.0 ECOBOOST 100 S&S EDITI...,FORD FIESTA V (2) 1.0 ECOBOOST 100 S&S EDITION 5P
166693,0.30,skoda octavia 3 III 2.0 TDI 150 GREEN TEC BUSI...,SKODA OCTAVIA III 2.0 TDI 150 GREEN TEC BUSINE...


### Quelques vérifications

In [26]:
matches['scraped_version'][166690]

'audi a4 (4e generation) avant IV (2) AVANT 2.0 TDI 177 BUSINESS LINE QUATTRO S TRONIC'

In [27]:
matches['Origional_data'][166690]

'AUDI A4 IV (2) AVANT 2.0 TDI 177 BUSINESS LINE QUATTRO S TRONIC'

### Suppression des doublons dans le match

In [42]:
matches_unique = matches.drop_duplicates()

## 5. Merge entre les datasets

Nous utilisons le datasets **matches_unique** pour lier entre ***data* et **scraped_data_makes**.

In [45]:
# 1er merge
first_merge = pd.merge(data, matches_unique, left_on='make_model_version', right_on='Origional_data')

In [47]:
data.shape

(166695, 11)

In [46]:
first_merge.shape

(166695, 14)

In [48]:
## 2eme merge
second_merge = pd.merge(first_merge, scraped_data_makes, left_on='scraped_version', right_on='model_version')

In [49]:
second_merge.shape

(166695, 79)

### Vérifications

In [50]:
second_merge[['Puissance_fiscale','Description']]

,Puissance_fiscale,Description
0,5 CV,"modele: 308 (2E GENERATION), version: 1.6 HDI FAP 92CH BUSINESS 5P, puissance_fiscale: 4, portes: 5.0, options: banquette arrière 3 places;volant cuir;cache bagages;airbag frontal;banquette 1/3 - 2/3;sièges rang 2 rabattables à plat;vitres teintées;boucliers av et ar couleur caisse;phares av. de jour à LED;airbags rideaux;rétroviseurs électriques et dégivrants;siège conducteur avec réglage lombaire;vitres électriques;feux de freinage d'urgence;banquette rabattable;feux ar. à LED;AFU;contrôle de pression des pneus;airbags front. + lat.;blanc banquise;lampes de lecture à l'arrière;sièges réglables en hauteur;alarme d'oubli d'extinction des feux;EBD;volant sport;température extérieure;système audio;becquet;feux automatiques;fermeture électrique;ESP;fermeture électrique automatique;bacs de portes avant;anti patinage;accoudoir central avant;commandes du système audio au volant;fixations ISOFIX;poches d'aumonières;lampes de lecture à l'avant;lunette arrière dégivrante;essuie-glace arrière;ordinateur de bord;phares halogènes;climatisation;volant réglable en hauteur et profondeur;siège conducteur réglable hauteur;boite à gants réfrigérée;ABS;ceintures avant ajustables en hauteur;prise 12V;ecran tactile multifonction;FAP;3 ceintures ar. 3 points;ouverture des vitres séquentielle, couleur: BLANC BANQUISE"
1,5 CV,"modele: 308 (2E GENERATION), version: 1.6 HDI FAP 92CH BUSINESS 5P, puissance_fiscale: 4, portes: 5.0, options: banquette arrière 3 places;volant cuir;cache bagages;airbag frontal;banquette 1/3 - 2/3;sièges rang 2 rabattables à plat;vitres teintées;boucliers av et ar couleur caisse;phares av. de jour à LED;airbags rideaux;rétroviseurs électriques et dégivrants;siège conducteur avec réglage lombaire;vitres électriques;feux de freinage d'urgence;banquette rabattable;feux ar. à LED;AFU;contrôle de pression des pneus;airbags front. + lat.;blanc banquise;lampes de lecture à l'arrière;sièges réglables en hauteur;alarme d'oubli d'extinction des feux;EBD;volant sport;température extérieure;système audio;becquet;feux automatiques;fermeture électrique;ESP;fermeture électrique automatique;bacs de portes avant;anti patinage;accoudoir central avant;commandes du système audio au volant;fixations ISOFIX;poches d'aumonières;lampes de lecture à l'avant;lunette arrière dégivrante;essuie-glace arrière;ordinateur de bord;phares halogènes;climatisation;volant réglable en hauteur et profondeur;siège conducteur réglable hauteur;boite à gants réfrigérée;ABS;ceintures avant ajustables en hauteur;prise 12V;ecran tactile multifonction;FAP;3 ceintures ar. 3 points;ouverture des vitres séquentielle, couleur: BLANC BANQUISE"
2,5 CV,"modele: 308 (2E GENERATION), version: 1.6 HDI FAP 92CH BUSINESS 5P, puissance_fiscale: 4, portes: 5.0, options: banquette arrière 3 places;volant cuir;cache bagages;airbag frontal;banquette 1/3 - 2/3;sièges rang 2 rabattables à plat;vitres teintées;boucliers av et ar couleur caisse;phares av. de jour à LED;airbags rideaux;rétroviseurs électriques et dégivrants;siège conducteur avec réglage lombaire;vitres électriques;feux de freinage d'urgence;banquette rabattable;feux ar. à LED;AFU;contrôle de pression des pneus;airbags front. + lat.;blanc banquise;lampes de lecture à l'arrière;sièges réglables en hauteur;alarme d'oubli d'extinction des feux;EBD;volant sport;température extérieure;système audio;becquet;feux automatiques;fermeture électrique;ESP;fermeture électrique automatique;bacs de portes avant;anti patinage;accoudoir central avant;commandes du système audio au volant;fixations ISOFIX;poches d'aumonières;lampes de lecture à l'avant;lunette arrière dégivrante;essuie-glace arrière;ordinateur de bord;phares halogènes;climatisation;volant réglable en hauteur et profondeur;siège conducteur réglable hauteur;boite à gants réfrigérée;ABS;ceintures avant ajustables en hauteur;prise 12V;ecran tactile multifonction;FAP;3 ceintures ar. 3 points;ouverture des vitres séquentielle, couleur: BLANC BANQ

### Suppression des colonnes redondantes

In [51]:
second_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166695 entries, 0 to 166694
Data columns (total 79 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Price                               166695 non-null  float64
 1   Make                                166695 non-null  object 
 2   Model                               166695 non-null  object 
 3   Model_year                          166695 non-null  int64  
 4   Mileage                             166695 non-null  object 
 5   Fuel                                166695 non-null  object 
 6   Gearbox                             166695 non-null  object 
 7   Online                              166695 non-null  object 
 8   Description                         166695 non-null  object 
 9   version                             166695 non-null  object 
 10  make_model_version                  166695 non-null  object 
 11  Match confidence (lower is

In [53]:
# Colonnes à supprimer
col_del = ['Match confidence (lower is better)', 'scraped_version', 'Origional_data','make_model_version',
'Unnamed: 0', 'model_version', 'year', 'brand', 'model']

second_merge = second_merge.drop(columns=col_del)

In [54]:
# Colonnes à renommer
col_rename = {'version_name':'version_name_scr', 'Price':'prix_occasion', 'prix':'prix_neuf'}
data_merged = second_merge.rename(columns=col_rename)

In [56]:
# vérification de prix neuf vs prix d'occasion
data_merged[['prix_neuf', 'prix_occasion', 'Model_year', 'Online']]

,prix_neuf,prix_occasion,Model_year,Online
0,21100,11220.0,2014,06/07/2018 à 3h47
1,21100,10352.0,2014,20/04/2018 à 13h20
2,21100,11446.0,2014,16/07/2018 à 3h33
3,21100,5491.0,2013,11/09/2018 à 15h21
4,21100,10626.0,2013,22/02/2018 à 11h56
...,...,...,...,...
166690,37700,18061.0,2012,08/05/2018 à 3h21
166691,33150,26612.0,2017,17/04/2018 à 12h29
166692,51190,55676.0,2012,02/02/2018 à 14h16
166693,15440,15749.0,2016,13/03/2018 à 19h12


### Sauvegarde 

In [55]:
data_merged.to_csv('../Datasets/Data_merged.csv')